In [ ]:
!pip install gdown

In [ ]:
!apt install unzip

In [ ]:
!gdown 1vqZl38YdCDijcFFp1cqMH-0FtA2s8KlK

In [ ]:
!gdown 1vYogKi-qNhYcZ3_ZFFS362rHmORpW0cx

In [ ]:
!gdown 1hwHxQ58-iMMNE8SEhThjE6wtO8fYGIxu

In [ ]:
!gdown 1fg7PdDR5D_K4T6olQlz1ONDJkc5CenhQ

In [ ]:
!gdown 1SKO-CHgujiqf0ZGCX9veI41ehmfPV6Pd

In [ ]:
!unzip -q children_face.zip
!unzip -q data_clean.zip
!unzip -q data1k.zip
!unzip -q data400.zip

In [ ]:
import os
os.mkdir('dataset')
os.mkdir('dataset/train')
os.mkdir('dataset/test')

In [ ]:
list_folder = []

root = 'data_aligned'
for label in os.listdir(root):
    try:
        list_file = os.listdir(f'{root}/{label}')
        if len(list_file)>5:
            list_folder.append(f'{root}/{label}')
    except:
        continue

root = "data_face_jun/train"
for label in os.listdir(root):
    try:
        list_file = os.listdir(f'{root}/{label}')
        if len(list_file)>5:
            list_folder.append(f'{root}/{label}')
    except:
        continue
root = 'data_face_jun/test'
for label in os.listdir(root):
    try:
        list_file = os.listdir(f'{root}/{label}')
        if len(list_file)>5:
            list_folder.append(f'{root}/{label}')
    except:
        continue
root = 'data1k'
for label in os.listdir(root):
    try:
        list_file = os.listdir(f'{root}/{label}')
        if len(list_file)>5:
            list_folder.append(f'{root}/{label}')
    except:
        continue
root = 'data400'
for label in os.listdir(root):
    try:
        list_file = os.listdir(f'{root}/{label}')
        if len(list_file)>5:
            list_folder.append(f'{root}/{label}')
    except:
        continue

In [ ]:
import random
random.shuffle(list_folder)
train_size = int(0.8*len(list_folder))
print(train_size)
list_train = list_folder[:train_size]
list_test = list_folder[train_size:]


In [ ]:
import shutil
for path in list_train:
    label = path.split('/')[-1]
    try:
        shutil.copytree(src = path,dst = f'dataset/train/{label}')
    except:
        continue
for path in list_test:
    label = path.split('/')[-1]
    try:
        shutil.copytree(src = path,dst = f'dataset/test/{label}')
    except:
        continue

In [ ]:
!pip install easydict
!pip install onnx2torch
!pip install tensorboard
!pip install matplotlib

In [ ]:
import glob
import os.path as osp
import random
import json
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
import os
import numpy as np
from torchvision import models,transforms
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import logging
import os
import sys
from torch import distributed
import math
from typing import Callable
import numpy as np
import argparse, os
from torchvision import transforms
from PIL import Image
from easydict import EasyDict as edict
import torch
import torch
from torch import distributed
from torch.nn.functional import linear, normalize
from onnx2torch import convert
import torch
import math
import logging    
import matplotlib.pyplot as plt
import os
import time
from typing import List

import torch
from torch.utils.tensorboard import SummaryWriter
from torch import distributed
rank = 0
local_rank = 0
world_size = 1
try:
    distributed.init_process_group(
        backend="nccl",
        init_method="tcp://127.0.0.1:12584",
        rank=rank,
        world_size=world_size,
    )
except:
    print('ok')

In [ ]:
resize = (112,112)
mean = [0.5,0.5,0.5]
std = [0.5,0.5,0.5]
transform = transforms.Compose([
          transforms.Resize(resize),
          transforms.ToTensor(),
          transforms.Normalize(mean,std),
        transforms.RandomHorizontalFlip(p=0.5),           # Lật ngang hình ảnh với xác suất 100%
    # transforms.ColorJitter(brightness=0.3,            # Thay đổi độ sáng
    #                        contrast=0.3),             # Thay đổi độ tương phản
    # transforms.GaussianBlur(kernel_size=5,            # Áp dụng làm mờ Gaussian
    #                         sigma=(0.1, 2.0)),  
        ])

In [ ]:
class Dataset(data.Dataset):
  def __init__(self,file_list):
    self.file_list = file_list
  def __len__(self):
    return len(self.file_list)
  def __getitem__(self, index):
    img_path = self.file_list[index]
    img = Image.open(img_path).convert('RGB')
    img_transformed = transform(img)
    label = get_label(img_path)
    return img_transformed,int(label)

In [ ]:
directory = 'dataset/train'
classes = os.listdir(directory)
def get_label(path):
  label_name = path.split('/')[-2]
  return classes.index(label_name)


X = []
for label in os.listdir(directory):
  for file_name in os.listdir(f'{directory}/{label}'):
    file_path = f'{directory}/{label}/{file_name}'
    X.append(file_path)
import random
random.shuffle(X)

In [ ]:
for path in tqdm(X):
    try:
        image = Image.open(path).convert('RGB')
    except:
        os.remove(path)
        X.remove(path)

In [ ]:
dataset = Dataset(X)

In [ ]:
batch_size = 128
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,shuffle = True,drop_last=True)

In [ ]:
dataloaders ={
    "train":train_loader
}

In [ ]:
batch_iterator = iter(dataloaders['train'])
inputs, labels = next(batch_iterator)
print(inputs.shape)
print(labels)

In [ ]:

class AverageMeter(object):
    """Computes and stores the average and current value
    """

    def __init__(self):
        self.val = None
        self.avg = None
        self.sum = None
        self.count = None
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def init_logging(rank, models_root):
    if rank == 0:
        log_root = logging.getLogger()
        log_root.setLevel(logging.INFO)
        formatter = logging.Formatter("Training: %(asctime)s-%(message)s")
        handler_file = logging.FileHandler(os.path.join(models_root, "training.log"))
        handler_stream = logging.StreamHandler(sys.stdout)
        handler_file.setFormatter(formatter)
        handler_stream.setFormatter(formatter)
        log_root.addHandler(handler_file)
        log_root.addHandler(handler_stream)
        log_root.info('rank_id: %d' % rank)


In [ ]:

class PartialFC_V2(torch.nn.Module):

    _version = 2

    def __init__(
        self,
        margin_loss: Callable,
        embedding_size: int,
        num_classes: int,
        sample_rate: float = 1.0,
        fp16: bool = False,
    ):

        super(PartialFC_V2, self).__init__()
        assert (
            distributed.is_initialized()
        ), "must initialize distributed before create this"
        self.rank = distributed.get_rank()
        self.world_size = distributed.get_world_size()

        self.dist_cross_entropy = DistCrossEntropy()
        self.embedding_size = embedding_size
        self.sample_rate: float = sample_rate
        self.fp16 = fp16
        self.num_local: int = num_classes // self.world_size + int(
            self.rank < num_classes % self.world_size
        )
        self.class_start: int = num_classes // self.world_size * self.rank + min(
            self.rank, num_classes % self.world_size
        )
        self.num_sample: int = int(self.sample_rate * self.num_local)
        self.last_batch_size: int = 0

        self.is_updated: bool = True
        self.init_weight_update: bool = True
        self.weight = torch.nn.Parameter(torch.normal(0, 0.01, (self.num_local, embedding_size)))

        # margin_loss
        if isinstance(margin_loss, Callable):
            self.margin_softmax = margin_loss
        else:
            raise

    def sample(self, labels, index_positive):

        with torch.no_grad():
            positive = torch.unique(labels[index_positive], sorted=True).cuda()
            if self.num_sample - positive.size(0) >= 0:
                perm = torch.rand(size=[self.num_local]).cuda()
                perm[positive] = 2.0
                index = torch.topk(perm, k=self.num_sample)[1].cuda()
                index = index.sort()[0].cuda()
            else:
                index = positive
            self.weight_index = index

            labels[index_positive] = torch.searchsorted(index, labels[index_positive])

        return self.weight[self.weight_index]

    def forward(
        self,
        local_embeddings: torch.Tensor,
        local_labels: torch.Tensor,
    ):

        local_labels.squeeze_()
        local_labels = local_labels.long()

        batch_size = local_embeddings.size(0)
        if self.last_batch_size == 0:
            self.last_batch_size = batch_size
        assert self.last_batch_size == batch_size, (
            f"last batch size do not equal current batch size: {self.last_batch_size} vs {batch_size}")

        _gather_embeddings = [
            torch.zeros((batch_size, self.embedding_size)).cuda()
            for _ in range(self.world_size)
        ]
        _gather_labels = [
            torch.zeros(batch_size).long().cuda() for _ in range(self.world_size)
        ]
        _list_embeddings = AllGather(local_embeddings, *_gather_embeddings)
        distributed.all_gather(_gather_labels, local_labels)

        embeddings = torch.cat(_list_embeddings)
        labels = torch.cat(_gather_labels)

        labels = labels.view(-1, 1)
        index_positive = (self.class_start <= labels) & (
            labels < self.class_start + self.num_local
        )
        labels[~index_positive] = -1
        labels[index_positive] -= self.class_start

        if self.sample_rate < 1:
            weight = self.sample(labels, index_positive)
        else:
            weight = self.weight

        with torch.cuda.amp.autocast(self.fp16):
            norm_embeddings = normalize(embeddings)
            norm_weight_activated = normalize(weight)
            logits = linear(norm_embeddings, norm_weight_activated)
        if self.fp16:
            logits = logits.float()
        logits = logits.clamp(-1, 1)

        logits = self.margin_softmax(logits, labels)
        loss = self.dist_cross_entropy(logits, labels)
        return loss


class DistCrossEntropyFunc(torch.autograd.Function):

    @staticmethod
    def forward(ctx, logits: torch.Tensor, label: torch.Tensor):
        """ """
        batch_size = logits.size(0)
        # for numerical stability
        max_logits, _ = torch.max(logits, dim=1, keepdim=True)
        # local to global
        distributed.all_reduce(max_logits, distributed.ReduceOp.MAX)
        logits.sub_(max_logits)
        logits.exp_()
        sum_logits_exp = torch.sum(logits, dim=1, keepdim=True)
        # local to global
        distributed.all_reduce(sum_logits_exp, distributed.ReduceOp.SUM)
        logits.div_(sum_logits_exp)
        index = torch.where(label != -1)[0]
        # loss
        loss = torch.zeros(batch_size, 1, device=logits.device)
        loss[index] = logits[index].gather(1, label[index])
        distributed.all_reduce(loss, distributed.ReduceOp.SUM)
        ctx.save_for_backward(index, logits, label)
        return loss.clamp_min_(1e-30).log_().mean() * (-1)

    @staticmethod
    def backward(ctx, loss_gradient):
        (
            index,
            logits,
            label,
        ) = ctx.saved_tensors
        batch_size = logits.size(0)
        one_hot = torch.zeros(
            size=[index.size(0), logits.size(1)], device=logits.device
        )
        one_hot.scatter_(1, label[index], 1)
        logits[index] -= one_hot
        logits.div_(batch_size)
        return logits * loss_gradient.item(), None


class DistCrossEntropy(torch.nn.Module):
    def __init__(self):
        super(DistCrossEntropy, self).__init__()

    def forward(self, logit_part, label_part):
        return DistCrossEntropyFunc.apply(logit_part, label_part)


class AllGatherFunc(torch.autograd.Function):
    """AllGather op with gradient backward"""

    @staticmethod
    def forward(ctx, tensor, *gather_list):
        gather_list = list(gather_list)
        distributed.all_gather(gather_list, tensor)
        return tuple(gather_list)

    @staticmethod
    def backward(ctx, *grads):
        grad_list = list(grads)
        rank = distributed.get_rank()
        grad_out = grad_list[rank]

        dist_ops = [
            distributed.reduce(grad_out, rank, distributed.ReduceOp.SUM, async_op=True)
            if i == rank
            else distributed.reduce(
                grad_list[i], i, distributed.ReduceOp.SUM, async_op=True
            )
            for i in range(distributed.get_world_size())
        ]
        for _op in dist_ops:
            _op.wait()

        grad_out *= len(grad_list)  # cooperate with distributed loss function
        return (grad_out, *[None for _ in range(len(grad_list))])


AllGather = AllGatherFunc.apply

class CombinedMarginLoss(torch.nn.Module):
    def __init__(self,
                 s,
                 m1,
                 m2,
                 m3,
                 interclass_filtering_threshold=0):
        super().__init__()
        self.s = s
        self.m1 = m1
        self.m2 = m2
        self.m3 = m3
        self.interclass_filtering_threshold = interclass_filtering_threshold

        # For ArcFace
        self.cos_m = math.cos(self.m2)
        self.sin_m = math.sin(self.m2)
        self.theta = math.cos(math.pi - self.m2)
        self.sinmm = math.sin(math.pi - self.m2) * self.m2
        self.easy_margin = False


    def forward(self, logits, labels):
        index_positive = torch.where(labels != -1)[0]

        if self.interclass_filtering_threshold > 0:
            with torch.no_grad():
                dirty = logits > self.interclass_filtering_threshold
                dirty = dirty.float()
                mask = torch.ones([index_positive.size(0), logits.size(1)], device=logits.device)
                mask.scatter_(1, labels[index_positive], 0)
                dirty[index_positive] *= mask
                tensor_mul = 1 - dirty
            logits = tensor_mul * logits

        target_logit = logits[index_positive, labels[index_positive].view(-1)]

        if self.m1 == 1.0 and self.m3 == 0.0:
            with torch.no_grad():
                target_logit.arccos_()
                logits.arccos_()
                final_target_logit = target_logit + self.m2
                logits[index_positive, labels[index_positive].view(-1)] = final_target_logit
                logits.cos_()
            logits = logits * self.s

        elif self.m3 > 0:
            final_target_logit = target_logit - self.m3
            logits[index_positive, labels[index_positive].view(-1)] = final_target_logit
            logits = logits * self.s
        else:
            raise

        return logits

class ArcFace(torch.nn.Module):
    def __init__(self, s=64.0, margin=0.5):
        super(ArcFace, self).__init__()
        self.s = s
        self.margin = margin
        self.cos_m = math.cos(margin)
        self.sin_m = math.sin(margin)
        self.theta = math.cos(math.pi - margin)
        self.sinmm = math.sin(math.pi - margin) * margin
        self.easy_margin = False


    def forward(self, logits: torch.Tensor, labels: torch.Tensor):
        index = torch.where(labels != -1)[0]
        target_logit = logits[index, labels[index].view(-1)]

        with torch.no_grad():
            target_logit.arccos_()
            logits.arccos_()
            final_target_logit = target_logit + self.margin
            logits[index, labels[index].view(-1)] = final_target_logit
            logits.cos_()
        logits = logits * self.s
        return logits
from torch.optim.lr_scheduler import _LRScheduler
from torch.optim import SGD
import torch
import warnings

class PolynomialLRWarmup(_LRScheduler):
    def __init__(self, optimizer, warmup_iters, total_iters=5, power=1.0, last_epoch=-1, verbose=False):
        super().__init__(optimizer, last_epoch=last_epoch, verbose=verbose)
        self.total_iters = total_iters
        self.power = power
        self.warmup_iters = warmup_iters


    def get_lr(self):
        if not self._get_lr_called_within_step:
            warnings.warn("To get the last learning rate computed by the scheduler, "
                          "please use `get_last_lr()`.", UserWarning)

        if self.last_epoch == 0 or self.last_epoch > self.total_iters:
            return [group["lr"] for group in self.optimizer.param_groups]

        if self.last_epoch <= self.warmup_iters:
            return [base_lr * self.last_epoch / self.warmup_iters for base_lr in self.base_lrs]
        else:
            l = self.last_epoch
            w = self.warmup_iters
            t = self.total_iters
            decay_factor = ((1.0 - (l - w) / (t - w)) / (1.0 - (l - 1 - w) / (t - w))) ** self.power
        return [group["lr"] * decay_factor for group in self.optimizer.param_groups]

    def _get_closed_form_lr(self):

        if self.last_epoch <= self.warmup_iters:
            return [
                base_lr * self.last_epoch / self.warmup_iters for base_lr in self.base_lrs]
        else:
            return [
                (
                    base_lr * (1.0 - (min(self.total_iters, self.last_epoch) - self.warmup_iters) / (self.total_iters - self.warmup_iters)) ** self.power
                )
                for base_lr in self.base_lrs
            ]


class CallBackLogging(object):
    def __init__(self, frequent, total_step, batch_size, start_step=0,writer=None):
        self.frequent: int = frequent
        self.rank: int = distributed.get_rank()
        self.world_size: int = distributed.get_world_size()
        self.time_start = time.time()
        self.total_step: int = total_step
        self.start_step: int = start_step
        self.batch_size: int = batch_size
        self.writer = writer

        self.init = False
        self.tic = 0

    def __call__(self,
                 global_step: int,
                 loss: AverageMeter,
                 epoch: int,
                 fp16: bool,
                 learning_rate: float,
                 grad_scaler: torch.cuda.amp.GradScaler):
        if self.rank == 0 and global_step > 0 and global_step % self.frequent == 0:
            if self.init:
                try:
                    speed: float = self.frequent * self.batch_size / (time.time() - self.tic)
                    speed_total = speed * self.world_size
                except ZeroDivisionError:
                    speed_total = float('inf')

                #time_now = (time.time() - self.time_start) / 3600
                #time_total = time_now / ((global_step + 1) / self.total_step)
                #time_for_end = time_total - time_now
                time_now = time.time()
                time_sec = int(time_now - self.time_start)
                time_sec_avg = time_sec / (global_step - self.start_step + 1)
                eta_sec = time_sec_avg * (self.total_step - global_step - 1)
                time_for_end = eta_sec/3600
                if self.writer is not None:
                    self.writer.add_scalar('time_for_end', time_for_end, global_step)
                    self.writer.add_scalar('learning_rate', learning_rate, global_step)
                    self.writer.add_scalar('loss', loss.avg, global_step)
                if fp16:
                    msg = "Speed %.2f samples/sec   Loss %.4f   LearningRate %.6f   Epoch: %d   Global Step: %d   " \
                          "Fp16 Grad Scale: %2.f   Required: %1.f hours" % (
                              speed_total, loss.avg, learning_rate, epoch, global_step,
                              grad_scaler.get_scale(), time_for_end
                          )
                else:
                    msg = "Speed %.2f samples/sec   Loss %.4f   LearningRate %.6f   Epoch: %d   Global Step: %d   " \
                          "Required: %1.f hours" % (
                              speed_total, loss.avg, learning_rate, epoch, global_step, time_for_end
                          )
                logging.info(msg)
                loss.reset()
                self.tic = time.time()
            else:
                self.init = True
                self.tic = time.time()



In [ ]:
from tqdm import tqdm
import numpy as np
import argparse, os
from torchvision import transforms
from PIL import Image
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def l2_norm(x, axis=1):
    """l2 norm"""
    norm = np.linalg.norm(x, axis=axis, keepdims=True)
    output = x / norm
    return output

def postprocess(onnx_output):
    return l2_norm(onnx_output)
def compute_FPR(gt, pred):
    FP = ((pred & ~gt)).sum()
    TN = (~pred & ~gt).sum()
    FPR = (FP / (FP + TN)).item()
    return FPR
def renorm(image):
    image = image.astype(np.float32)
    image = (image - image.min()) / (image.max() - image.min() + 1e-4)
    image = (255 * image).astype(np.uint8)
    return image

def unnormalize(image, mean, std):
    image = ((image * torch.as_tensor(std).reshape(1, image.size(1), 1, 1).to(image.device)) + torch.as_tensor(mean).reshape(1, image.size(1), 1, 1).to(image.device))
    return image
metric = torch.nn.CosineSimilarity(dim=-1)
def eval_compare(backbone,mode):
    backbone.eval()
    embedding_1, embedding_2, mapping = list(), list(), dict()
    for idx, cls_name in enumerate(tqdm(os.listdir(os.path.join('dataset', mode)))):
        front_list, portrait_list = list(), list()
        list_path =  os.listdir(os.path.join('dataset', mode, cls_name))
        if len(list_path)>1:
          fname_1 = list_path[0]
          fname_2 = list_path[1]
          image_1 = Image.open(os.path.join('dataset', mode, cls_name, fname_1)).convert('RGB')
          image_2 = Image.open(os.path.join('dataset', mode, cls_name, fname_2)).convert('RGB')
          image_1 = Image.fromarray(renorm(np.array(image_1)))
          image_2 = Image.fromarray(renorm(np.array(image_2)))
    
          tensor_1 = transform(image_1)
          tensor_2 = transform(image_2)
          inputs = torch.stack([tensor_1, tensor_2], dim=0).to(device)
          with torch.no_grad():
            embed = backbone(inputs)
          embedding_1.append(embed[0])
          embedding_2.append(embed[1])
          mapping[idx] = cls_name
    embedding_1 = torch.stack(embedding_1, dim=0)
    embedding_2 = torch.stack(embedding_2, dim=0)
    
    scores = list()
    for e in embedding_1:
        scores.append(metric(e.unsqueeze(dim=0), embedding_2))
    scores = torch.stack(scores, dim=0)
    desc_scores, indices = torch.sort(scores, dim=1, descending=True)
    
    thresholds = np.arange(0.1, 1.0, 0.025)
    acc, max_fpr, avg_fpr = list(), list(), list()
    
    for thres in thresholds:
        FPR, outputs = list(), list()
        for q, (indice, score) in enumerate(zip(indices, desc_scores)):
            # define query ground-truth like [0, 0, 1, 0, ...]
            gt = torch.zeros_like(score)
            gt[(indice == q).nonzero().item()] = 1
            gt = gt.bool()
    
            # compute false positive rate
            pred = score >= thres
            FPR.append(compute_FPR(gt, pred))
    
            if score[indice == q] >= thres:
                outputs.append(True)
            else:
                outputs.append(False)
    
        acc.append(np.sum(outputs) / len(outputs))
        max_fpr.append(np.amax(FPR))
        avg_fpr.append(np.mean(FPR))


    # Vẽ biểu đồ
    plt.plot(thresholds, acc, label='Recall', marker='o')
    plt.plot(thresholds, max_fpr, label='FPR', marker='x')
    plt.plot(thresholds, avg_fpr, label='Avg FPR', marker='s')
    
    # Đặt nhãn và tiêu đề
    plt.xlabel('Threshold')
    plt.ylabel('Value')
    plt.title(f'Eval face compare {mode}')
    plt.legend()
    
    # Hiển thị biểu đồ
    plt.grid(True)
    plt.show()
def get_face_embedding_list(backbone,faces_pil):
    tensors = [transform(face_pil) for face_pil in faces_pil]
    inputs = torch.stack(tensors, dim=0).to(device)
    with torch.no_grad():
      embeds = backbone(inputs).cpu().detach().numpy()
      return np.array([postprocess(np.expand_dims(embed,0)) for embed  in embeds])
def eval_search(backbone,mode):
    backbone.eval()
    face_ids_map = {}
    sources_db = np.empty((0, 512), dtype=np.float32)
    targets_db = np.empty((0, 512), dtype=np.float32)
    face_ids = []
    classes = []
    for cls_name in tqdm(os.listdir(f'dataset/{mode}')):
        list_file = [f'dataset/{mode}/{cls_name}/{file_name}' for file_name in os.listdir(f'dataset/{mode}/{cls_name}')]
        sources = list_file[:int(len(list_file)/2)][:1]
        targets = list_file[int(len(list_file)/2):][:1]
        if len(sources)>0 and len(targets)>0:
            sources_emb = get_face_embedding_list(backbone,[Image.open(source).convert('RGB') for source in sources])
            targets_emb =  get_face_embedding_list(backbone,[Image.open(target).convert('RGB') for target in targets])
            for source_emb in sources_emb:
              for target_emb in targets_emb:
                sources_db = np.concatenate([sources_db, source_emb], axis=0)
                targets_db = np.concatenate([targets_db,target_emb], axis=0)
                classes.append(cls_name)
                if cls_name not in face_ids_map:
                    face_ids_map[cls_name] = len(face_ids_map)
                face_id = face_ids_map[cls_name]
                face_ids.append(face_id)
    face_ids = np.array(face_ids)
    distance = [np.sum(sources_db * targets_db, axis=1)]
    label = [face_ids == face_ids]
    num_of_sample = len(os.listdir(f'dataset/{mode}'))
    for _ in tqdm(range(num_of_sample)):
        ids = np.arange(sources_db.shape[0])
        np.random.shuffle(ids)
        distance.append(np.sum(sources_db[ids] * targets_db, axis=1))
        label.append(face_ids[ids] == face_ids)
    distance = np.concatenate(distance, axis=0)
    label = np.concatenate(label, axis=0)
    # cal TRec, FRec, Frec_, TRej
    threshs = np.linspace(0., 1.0, 101)
    results = np.empty((threshs.shape[0], 2), dtype=np.float32)
    for i, thresh in enumerate(threshs):
        pos_id = distance > thresh
    
        TRec = np.sum(pos_id[pos_id] == label[pos_id])
        FRec = np.sum(pos_id) - TRec
        FRej_Pos = np.sum(label) - TRec
        # negative
        FRec_Neg = np.sum(pos_id[np.logical_not(label)])
        TRej = label[np.logical_not(label)].shape[0] - FRec_Neg
    
        results[i, 0] = TRec / np.sum(label)  # TRecR
        results[i, 1] = TRec / (TRec + FRec + FRec_Neg) if (TRec + FRec + FRec_Neg) > 0 else 0  # Prec

    fig, ax = plt.subplots()
    
    ax.plot(threshs, results[:, 0], label=f"TRecR_0")
    ax.plot(threshs, results[:, 1], label=f"Prec_0")
    ax.set(xlabel='thresh', ylabel='score (cosine similarity)')
    ax.grid()
    plt.legend()
    plt.title(f'Eval search {mode}')
    plt.show()

In [ ]:
model_name = 'mbf_w600k_fine_tune'

margin_list = (1.0, 0.0, 0.4)
interclass_filtering_threshold=0
margin_loss = CombinedMarginLoss(
    64,
    margin_list[0],
    margin_list[1],
    margin_list[2],
    interclass_filtering_threshold
)
cfg = edict()
cfg.margin_list = (1.0, 0.0, 0.4)
cfg.network = model_name
cfg.resume = False
cfg.output = f'{model_name}/'
cfg.embedding_size = 512
cfg.sample_rate = 1.0
cfg.fp16 = True
cfg.weight_decay = 1e-4
cfg.batch_size = 128
cfg.optimizer = "adamw"
cfg.lr = 0.00001
cfg.verbose = 2000
cfg.dali = True
cfg.dali_aug = True

cfg.num_workers = 6

cfg.rec = "dataset"
cfg.num_classes = len(classes)
cfg.num_image = len(X)
cfg.num_epoch = 20
cfg.warmup_epoch = 2
cfg.gradient_acc = 1
cfg.frequent = 10
cfg.val_targets = []



In [ ]:
module_partial_fc = PartialFC_V2(
            margin_loss, cfg.embedding_size, cfg.num_classes,
            cfg.sample_rate, False)
module_partial_fc.train().cuda()

In [ ]:
backbone = convert('w600k_mbf.onnx').to(device)

In [ ]:
list_params_name = [
    "Gemm_96.weight","Gemm_96.bias",
    "BatchNormalization_97.weight","BatchNormalization_97.bias"
]
list_params = []
for name, param in backbone.named_parameters():
    if name not in list_params_name:
        param.requires_grad = False
    else:
        param.requires_grad = True
        list_params.append(param)

In [ ]:
opt = torch.optim.AdamW(params=[
    # {'params': backbone.parameters()},
    {'params': backbone.Gemm_96.parameters()},
    {'params': backbone.BatchNormalization_97.parameters()},
# {'params': backbone.Conv_93.parameters()}
],
    lr=cfg.lr, weight_decay=cfg.weight_decay)

cfg.total_batch_size = cfg.batch_size * world_size
cfg.warmup_step = cfg.num_image // cfg.total_batch_size * cfg.warmup_epoch
cfg.total_step = cfg.num_image // cfg.total_batch_size * cfg.num_epoch

lr_scheduler = PolynomialLRWarmup(
    optimizer=opt,
    warmup_iters=cfg.warmup_step,
    total_iters=cfg.total_step)
loss_am = AverageMeter()
amp = torch.cuda.amp.grad_scaler.GradScaler(growth_interval=100)

In [ ]:
summary_writer = (
    SummaryWriter(log_dir=os.path.join(cfg.output, "tensorboard"))
    if rank == 0
    else None
)
start_epoch = 0
global_step = 0
callback_logging = CallBackLogging(
    frequent=cfg.frequent,
    total_step=cfg.total_step,
    batch_size=cfg.batch_size,
    start_step = global_step,
    writer=summary_writer
)
best_loss = 1000
eval_compare(backbone,'train')
eval_search(backbone,'train')
eval_compare(backbone,'test')
eval_search(backbone,'test')
print('-------------------------------------------------------------------------------------')
for epoch in range(start_epoch, 20):
    running_loss = 0.0
    for phase in ['train', 'test']:
        if phase =='train':
            backbone.train()
            module_partial_fc.train()
            with torch.set_grad_enabled(phase == 'train'):
                with tqdm(dataloaders[phase],unit="batch") as tepoch:
                  for img, local_labels in tepoch:
                      tepoch.set_description(f"Epoch {epoch}")
                      img = img.to(device)
                      local_labels = local_labels.to(device)
                      global_step += 1
                      local_embeddings = backbone(img)
                      loss: torch.Tensor = module_partial_fc(local_embeddings, local_labels)
                      running_loss += loss.item()
                      loss.backward()
                        # params_to_clip = list(backbone.Flatten_95.parameters()) + list(backbone.Gemm_96.parameters()) + list(backbone.BatchNormalization_97.parameters()) +list(backbone.PRelu_94.parameters())+list(backbone.Conv_93.parameters())
                        # torch.nn.utils.clip_grad_norm_(params_to_clip, max_norm=5.0)
                      opt.step()
                      opt.zero_grad()
                      lr_scheduler.step()
                      with torch.no_grad():
                          loss_am.update(loss.item(), 1)
                          callback_logging(global_step, loss_am, epoch, cfg.fp16, lr_scheduler.get_last_lr()[0], amp)
                      tepoch.set_postfix(loss=loss.item())
                  epoch_loss = running_loss / len(dataloaders[phase])
                  print('{}: Loss: {:.4f}'.format(phase,epoch_loss))
                  path_module = os.path.join(cfg.output, f"{epoch}_model.pt")
                  path_fc = os.path.join(cfg.output, f"{epoch}_fc_model.pt")
                  torch.save(backbone.state_dict(), path_module)
                  torch.save(module_partial_fc.state_dict(), path_fc)
        eval_compare(backbone,phase)
        eval_search(backbone,phase)
        print('-------------------------------------------------------------------------------------')

In [ ]:
summary_writer = (
    SummaryWriter(log_dir=os.path.join(cfg.output, "tensorboard"))
    if rank == 0
    else None
)
start_epoch = 0
global_step = 0
callback_logging = CallBackLogging(
    frequent=cfg.frequent,
    total_step=cfg.total_step,
    batch_size=cfg.batch_size,
    start_step = global_step,
    writer=summary_writer
)
best_loss = 1000
eval_compare(backbone,'train')
eval_search(backbone,'train')
eval_compare(backbone,'test')
eval_search(backbone,'test')
print('-------------------------------------------------------------------------------------')
for epoch in range(start_epoch, 20):
    running_loss = 0.0
    for phase in ['train', 'test']:
        if phase =='train':
            backbone.train()
            module_partial_fc.train()
            with torch.set_grad_enabled(phase == 'train'):
                with tqdm(dataloaders[phase],unit="batch") as tepoch:
                  for img, local_labels in tepoch:
                      tepoch.set_description(f"Epoch {epoch}")
                      img = img.to(device)
                      local_labels = local_labels.to(device)
                      global_step += 1
                      local_embeddings = backbone(img)
                      loss: torch.Tensor = module_partial_fc(local_embeddings, local_labels)
                      running_loss += loss.item()
                      loss.backward()
                        # params_to_clip = list(backbone.Flatten_95.parameters()) + list(backbone.Gemm_96.parameters()) + list(backbone.BatchNormalization_97.parameters()) +list(backbone.PRelu_94.parameters())+list(backbone.Conv_93.parameters())
                        # torch.nn.utils.clip_grad_norm_(params_to_clip, max_norm=5.0)
                      opt.step()
                      opt.zero_grad()
                      lr_scheduler.step()
                      with torch.no_grad():
                          loss_am.update(loss.item(), 1)
                          callback_logging(global_step, loss_am, epoch, cfg.fp16, lr_scheduler.get_last_lr()[0], amp)
                      tepoch.set_postfix(loss=loss.item())
                  epoch_loss = running_loss / len(dataloaders[phase])
                  print('{}: Loss: {:.4f}'.format(phase,epoch_loss))
                  path_module = os.path.join(cfg.output, f"{epoch}_model.pt")
                  path_fc = os.path.join(cfg.output, f"{epoch}_fc_model.pt")
                  torch.save(backbone.state_dict(), path_module)
                  torch.save(module_partial_fc.state_dict(), path_fc)
        eval_compare(backbone,phase)
        eval_search(backbone,phase)
        print('-------------------------------------------------------------------------------------')